
# Sensitivities: Bump vs AAD 

This dashboard demonstrates the accuracy and performance of AAD sensitivities for a range of complex products
- European Option
- Barrier Option
- Accumulator
- Asian Basket Option
- TaRF

all represented as scripted trades.

We are running ORE twice, with differences in the pricinengine.xml file:
- setting UseAD to false for the bump run and
- setting UseAD to true for AAD.

## Run ORE "Classic", Bump Sensitivities, Single-Threaded

In [ ]:
from ORE import *
import sys, time, math
sys.path.append('..')
import utilities

In [ ]:
params = Parameters()
params.fromFile("Input/ore.xml")
ore = OREApp(params)
ore.run()
utilities.checkErrorsAndRunTime(ore)
#utilities.writeList(ore.getReportNames())

## Run Again with ComputationGraph Enabled

In [ ]:
params_cg = Parameters()
params_cg.fromFile("Input/ore_cg.xml")
ore_cg = OREApp(params_cg)
ore_cg.run()
utilities.checkErrorsAndRunTime(ore_cg)
#utilities.writeList(ore_bump.getReportNames())

In [ ]:
# Compare Pricing Stats
utilities.match_pricingstats_12(ore, ore_cg, "Speedup", True)

In [ ]:
# Compare Sensitivities
utilities.match_sensi_reports(ore, ore_cg, "DeltaBump", "DeltaCG", False)

## Run Again using AAD

In [ ]:
params_ad = Parameters()
params_ad.fromFile("Input/ore_ad.xml")
ore_ad = OREApp(params_ad)
ore_ad.run()
utilities.checkErrorsAndRunTime(ore_ad)
#utilities.writeList(ore_ad.getReportNames())

In [ ]:
# Compare Pricing Stats
utilities.match_pricingstats_12(ore, ore_ad, "Speedup", False)

In [ ]:
# Compare NPV reports, expecting identical results
npv = ore.getReport("npv")
npv_ad = ore_ad.getReport("npv")
print("Bump & Reval NPV Report:")
display(utilities.format_report(npv))
print("AD NPV Report:")
display(utilities.format_report(npv_ad))

In [ ]:
# Compare Sensitivities
utilities.match_sensi_reports(ore, ore_ad, "DeltaBump", "DeltaAD", False)

## Discussion
Impact of:
- Shift Scheme (EQ Spot)
- Shift Size (Vega)
- Lowering the BarrierLevel in EquityBarrierOption
- Lowering the KnockOutLevel in EquityAccumulator
- Lowering the KnockOutProfitAmount in FxTARF, introducing varying range strikes or leverages

Todo:
- Add CAM

# Run again using the external GPU device

Note that this functionality is work in progress.

Using an external device requires another few changes in pricingengine.xml's EngineParameters section:
- set **UseCG** to true
- set **UseExternalComputeDevice** to true
- set **ExternalComputeDevice** to the device name (**OpenCL/Apple/Apple M2 Max** on my machine, a **38 core GPU** device)

Determine your available compute devices by running the QuantExt test suite with **quantext-test-suite --log_level=message --run_test="*/ComputeEnvironmentTest/testEnvironmentInit"**

You should also see the device name **BasicCpu/Default/Default** which is the CPU and can be used for testing.

In [ ]:
params_gpu = Parameters()
params_gpu.fromFile("Input/ore_gpu.xml")
ore_gpu = OREApp(params_gpu)
ore_gpu.run()
utilities.checkErrorsAndRunTime(ore_gpu)
#utilities.writeList(ore_ad.getReportNames())

In [ ]:
# Compare Pricing Stats
utilities.match_pricingstats_123(ore, ore_ad, ore_gpu, "SpeedupAD", "SpeedupGPU", False)

In [ ]:
# Compare Sensitivities
utilities.match_sensi_reports(ore, ore_gpu, "DeltaBump", "DeltaGPU", False)

### Todos
- debug the BasicCpu device calculation
- double precision for the OpenCL implementation
- random number generators for the OpenCL implementation (MT antithetic, Sobol, Sobol BB)
- CUDA implementation